# Docking Analysis

For both RDKit (rdkit_250_etkdg) and DMCG (dmcg_250) we generate these small ensembles:
 * 1 unminimized (single_unranked_unmin)
 * 1 unbiased sampled minimized (same as above - single_unranked_min)
 * 1 minimal energy (single_eranked_min)
 * 5 unbiased sampled energy minimized (5_unranked_min)
 * 5 sorted by energy and filtered by RMSDs: 0, 0.5, 1.0, 2.0 (5_filtered_R_min)
 
Each of these is docked.  The single minimal energy pose is also docked with 5x the exhaustiveness as a reference for the 5 sized ensembles.

In [1]:
import re, sys, glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from rdkit.Chem import AllChem as Chem
import gzip
from matplotlib import cm
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
import matplotlib.transforms as transforms


In [2]:
dkind = 'docked'

In [3]:
data = []
errors = []
for fname in glob.glob(f'wierbowski_cd/*/*_{dkind}_*.rms.txt') + glob.glob(f'refined-set/*/*_{dkind}.rms.txt'):
    if 'tmp' in fname:
        continue
    m = re.search(r'(\S+)/(\S+)/(\S+)_(rdkit|dmcg)_250_(etkdg_|dg_)?2dock_(5|single)_(eranked|unranked|filtered_\S+)_(min|unmin)_(e5_|s5_)?s?docked(_\S+)?.rms.txt',fname)
    bench,target,prefix,generator,rdgen,size,ranking,minimized,exhaustiveness,pdb = m.groups()
    sdf = fname.replace('.rms.txt','.sdf.gz')
    num = 0
    with gzip.open(sdf) as sdffile:
        supp = Chem.ForwardSDMolSupplier(sdffile)
        if dkind == 'docked':
            mols = [mol.GetProp('CNNscore') for mol in supp]
        else:
            mols = [-float(mol.GetProp('minimizedAffinity')) for mol in supp]
        if len(mols) == 0:
            errors.append(fname)
            continue
        idx = np.argmax(mols)
        num = len(mols)
    
    rmsds = pd.read_csv(fname,delim_whitespace=True,usecols=(2,),names=['rmsd'])
    if len(rmsds) == 0:
        errors.append(fname)
        continue
    rmsd = rmsds.iloc[idx].rmsd

        
    thresh = 0
    m = re.search(r'filtered_(\S+)',ranking)
    if m:
        thresh = float(m.group(1))
        
    if 'single' in size:
        size = 1
    else:
        size = int(size)
        
    if exhaustiveness == 'e5_':
        exhaustiveness = 40
    elif exhaustiveness == 's5_':
        size = 5
        exhaustiveness = 8
        ranking = 'same'
    elif exhaustiveness == None:
        exhaustiveness = 8
    else:
        assert 'Unknown exhaustiveness'
        
    minimized = minimized == 'min'
    
    if pdb:
        pdb = pdb.lstrip('_')
        
    data.append([bench,target,pdb,prefix,generator,size,ranking,minimized,exhaustiveness,thresh,rmsd,idx,num])

KeyError: 'CNNscore'

In [ ]:
fname

In [ ]:
errors

In [ ]:
D = pd.DataFrame(data,columns=('Benchmark','Target','Receptor','Ligand','Generator','Size','Ranking','Minimized','Exhaustiveness','Threshold','RMSD','BestIndex','NumConfs'))

Evaluate the distribution of docked pairs across targets.  Most are 100, so I think we are fine combining these all without worry too much about target bias.

In [ ]:
lines = pd.read_csv('wierbowski_cd/ds_cd_input_pairs.txt',delim_whitespace=True,names=('rec','lig','lig2','out'))
lines['targ'] = lines.rec.str.extract(r'\S+/(\S+)/\S+_PRO.pdb')
plt.hist(lines.groupby('targ').count().out);

In [ ]:
sum(lines.groupby('targ').count()['out'] ==100)/92

In [ ]:
#colors from bioactivity analysis

subset_sizes = [1,5,10,25,50,100,250]

colormaps = {
    'rdkitdg': cm.get_cmap('Greys'),
    'rdkitetkdg': cm.get_cmap('autumn_r'),
    'rdkit': cm.get_cmap('autumn_r'),
    'dmcg': cm.get_cmap('winter_r'),
    'auto3d': cm.get_cmap('Purples')
}

colors = {
    'rdkitdg': {N:colormaps['rdkitdg']((i+25)/(len(subset_sizes)+25)) for i,N in enumerate(subset_sizes)},
    'rdkitetkdg': {N:colormaps['rdkitetkdg'](i/len(subset_sizes)) for i,N in enumerate(subset_sizes)},
    'rdkit': {N:colormaps['rdkitetkdg'](i/len(subset_sizes)) for i,N in enumerate(subset_sizes)},
    'dmcg': {N:colormaps['dmcg'](i/len(subset_sizes)) for i,N in enumerate(subset_sizes)},
    'auto3d': {N:colormaps['auto3d']((i+2)/(len(subset_sizes)+2)) for i,N in enumerate(subset_sizes)}
}

names = {
    'platinum2017': "Platinum 2017",
    'refined-set': "PDBbind 2020 Refined",
    'rdkitdg': "RDKit DG",
    'rdkitetkdg': "RDKit ETKDGv3",
    'rdkit': "RDKit ETKDGv3",
    'dmcg': "DMCG",
    'auto3d':"Auto3D",
    'wierbowski_cd': 'Wierbowski CrossDock'
}

In [ ]:
D.groupby(['Benchmark','Generator','Size','Ranking','Minimized','Exhaustiveness','Threshold']).count()

Consider generating a single conformer with RDKit or DMCG and with or without minimization.

In [ ]:
        
for b in ['wierbowski_cd','refined-set']:
    plt.figure(figsize=(6,4),dpi=600)
    plt.axvline(2,color='gray',linestyle='--',linewidth=1)    
    bench = D[(D.Benchmark == b) & (D.Ranking == 'unranked') & (D.Exhaustiveness == 8) & (D.Size == 1)] 
    for a in ['rdkit','dmcg']:        
        for M in [False,True]:
                df = bench[(bench.Minimized==M) & (bench.Generator == a)]
                r = df.sort_values(by='RMSD').RMSD
                r = np.array(r)
                plt.plot([0]+list(r),np.linspace(0,1,len(r)+1),
                         linestyle='-' if M else '--',
                         alpha=0.5 if a == 'rdkit' else 1.0,
                         label=f'{names[a]}' if M else None, color=colors[a][1])
                
    plt.gca().add_line(Line2D([0], [0], color='black',label='Minimized'))
    plt.gca().add_line(Line2D([0], [0], linestyle='--', color='black',label='Unminimized'))
    plt.xlim(0,8)
    plt.ylim(0,1)
    plt.xlabel('RMSD of First Docking Pose')
    plt.ylabel(f'Fraction of {names[b]}')
    plt.legend();
    plt.savefig(f'{dkind}_single_unranked_{b}.pdf',bbox_inches='tight')

Best of 250 versus single sample

In [ ]:
        
for b in ['wierbowski_cd','refined-set']:
    plt.figure(figsize=(6,4),dpi=600)
    plt.axvline(2,color='gray',linestyle='--',linewidth=1)    
    bench = D[(D.Benchmark == b) & (D.Exhaustiveness == 8) & (D.Minimized) & (D.Size == 1)] 
    for a in ['rdkit','dmcg']:        
        for G in ['unranked','eranked']:
                df = bench[(bench.Ranking==G) & (bench.Generator == a)]
                r = df.sort_values(by='RMSD').RMSD
                r = np.array(r)
                plt.plot([0]+list(r),np.linspace(0,1,len(r)+1),                         
                         label=f'{names[a]} {"Single" if G == "unranked" else "Lowest Energy (250)"}', color=colors[a][1 if G == 'unranked' else 250])
                
    plt.xlim(0,8)
    plt.ylim(0,1)
    plt.xlabel('RMSD of First Docking Pose')
    plt.ylabel(f'Fraction of {names[b]}')
    plt.legend();
    plt.savefig(f'{dkind}_single_eranked_{b}.pdf',bbox_inches='tight')

In [ ]:
        
for b in ['wierbowski_cd','refined-set']:
    for a in ['rdkit','dmcg']:        

        plt.figure(figsize=(6,4),dpi=600)
        plt.axvline(2,color='gray',linestyle='--',linewidth=1)    
        bench = D[(D.Benchmark == b) & (D.Generator == a) & (D.Minimized)]
        
        df = bench[(bench.Size == 1) & (bench.Ranking == 'eranked') & (bench.Exhaustiveness == 8)]
        r = df.sort_values(by='RMSD').RMSD
        r = np.array(r)
        plt.plot([0]+list(r),np.linspace(0,1,len(r)+1),   
                 linewidth=1,
                label=f'Lowest Energy Conformer', color=colors[a][1])

#        df = bench[(bench.Size == 5) & (bench.Ranking == 'same') & (bench.Exhaustiveness == 8)]
#        r = df.sort_values(by='RMSD').RMSD
#        r = np.array(r)
#        plt.plot([0]+list(r),np.linspace(0,1,len(r)+1),   
#                label=f'Lowest Energy (5x)', color='k')
        
        df = bench[(bench.Size == 1) & (bench.Ranking == 'eranked') & (bench.Exhaustiveness == 40)]
        r = df.sort_values(by='RMSD').RMSD
        r = np.array(r)
        plt.plot([0]+list(r),np.linspace(0,1,len(r)+1),   
                label=f'Lowest Energy (5x Sampling)', color='purple',alpha=0.8)        
        
        for G in sorted(pd.unique(bench.Ranking)):
            
            df = bench[(bench.Ranking==G) & (bench.Size == 5)]
            if len(df) == 0:
                continue
            r = df.sort_values(by='RMSD').RMSD
            r = np.array(r)
            t = 0
            if 'filtered_0.5' in G:
                linestyle='dotted'
                t = 0.5
            elif 'filtered_1.0' in G:
                linestyle='dashdot'
                t = 1.0
            elif 'filtered_2.0' in G:
                linestyle = 'dashed'
                t = 2.0
            elif 'filtered' in G:
                linestyle = '-'
                continue
            else:         
                continue
                linestyle = '-'
            plt.plot([0]+list(r),np.linspace(0,1,len(r)+1),       
                     linewidth=1, color=colors[a][5], linestyle=linestyle,
                     label=f'{t} RMSD Cutoff (5 Conformers)')

        plt.xlim(0,8)
        plt.ylim(0,1)
        plt.title(f'{names[a]}')
        plt.xlabel('RMSD of First Docking Pose')
        plt.ylabel(f'Fraction of {names[b]}')
        plt.legend();
        plt.savefig(f'{dkind}_multi_eranked_{a}_{b}.pdf',bbox_inches='tight')

In [ ]:
df = bench[(bench.Size == 1) & (bench.Ranking == 'eranked') & (bench.Exhaustiveness == 40)]
df

In [ ]:
D.groupby(['Benchmark','Generator','Size','Ranking','Minimized','Exhaustiveness','Threshold']).count()

Consider just the 2A level of accuracy

In [ ]:
bench = D[(D.Minimized)]


In [ ]:
bench.groupby(['Benchmark','Generator','Size','Ranking','Minimized','Exhaustiveness','Threshold']).count()

In [ ]:
bench = bench.copy()
bench.loc[:,'combo'] =  bench[['Size','Ranking','Exhaustiveness']].astype(str).apply(' '.join, axis=1)

Calculate bootstrap statistics

In [ ]:

bench.loc[:,'top1'] = (bench.RMSD <= 2).astype(int)

p = sns.catplot(data=bench, col_order=['refined-set','wierbowski_cd'], hue='Generator',col='Benchmark',\
                x='combo',y='top1',\
                order=['1 unranked 8', '1 eranked 8', '1 eranked 40', '5 unranked 8', \
                       '5 filtered_0 8','5 filtered_0.5 8', '5 filtered_1.0 8', '5 filtered_2.0 8'],\
                kind='bar',palette={'rdkit': colors['rdkit'][5], 'dmcg': colors['dmcg'][5]},
               legend=None, n_boot=1000, errwidth=2,linestyle='-',edgecolor='k',linewidth=0.5)


p.fig.set_figwidth(8)
p.fig.set_figheight(4)
p.fig.set_dpi(300)
p.axes[0,0].set_ylabel('Fraction <2Å RMSD',fontsize=14)
p.axes[0,0].set_xlabel(None)
p.axes[0,1].set_xlabel(None)

p.axes[0,0].set_title(names[p.axes[0,0].get_title().split()[-1]])
p.axes[0,1].set_title(names[p.axes[0,1].get_title().split()[-1]])

p.axes[0,0].errorbar

for i in range(3):
    for j in range(2):
        p.axes[0,j].patches[i+8].set_facecolor(colors['dmcg'][1])
        p.axes[0,j].patches[i].set_facecolor(colors['rdkit'][1])

p.axes[0,1].legend(handles=p.axes[0,1].get_legend_handles_labels()[0], 
                   labels=[names[l] for l in p.axes[0,1].get_legend_handles_labels()[1]],
                   loc='upper center',ncol=2)
p.set_xticklabels(['Random', 'Lowest\nEnergy', 'Lowest\nEnergy (5x)', 'Random', \
                       'Lowest\nEnergy','RMSD\nCutoff 0.5', 'RMSD\nCutoff 1.0', 'RMSD\nCutoff 2.0'],rotation=90)


#p.axes[0,0].set_clip_on(False)

for i in range(2):
    annotation = p.axes[0,i].annotate('Single Conformer',annotation_clip=False,xy=(1,-0.225),xytext=(1,-0.24),
                                  va='top', 
                                  ha='center',arrowprops={'arrowstyle':'-[, widthB=4, lengthB=0.5,angleB=0'})
    
    p.axes[0,i].annotate('5 Conformers',annotation_clip=False,xy=(5,-0.225),xytext=(5,-0.24),
                                  va='top',
                                  ha='center',arrowprops={'arrowstyle':'-[, widthB=6, lengthB=0.5,angleB=0'})

    
plt.savefig(f'{dkind}_2A.pdf',bbox_inches='tight')

In [ ]:
p = sns.catplot(data=bench[bench.combo=='1 eranked 8'], x='Benchmark', hue='Generator',y='top1',
                kind='bar',palette={'rdkit': '#003594', 'dmcg': '#FFB81C'},
                legend=None,n_boot=1000, errwidth=2,linestyle='-',edgecolor='k',linewidth=0.5)

p.fig.set_figwidth(4)
p.fig.set_figheight(4)
p.fig.set_dpi(300)
a = p.axes[0,0]
a.set_ylabel('Fraction <2Å RMSD',fontsize=16)
a.set_xlabel(None)
a.set_xticklabels(['Crossdocking','Redocking'],fontsize=14)
a.legend(handles=[h[0] for h in a.get_legend_handles_labels()[0]], 
                   labels=['RDKit','DMCG'],
                   loc='upper left',fontsize=14)

In [ ]:
a.get_legend_handles_labels()[0][1][0]

In [ ]:
grouped = bench.groupby(['Benchmark','Generator','combo'])

In [ ]:
from scipy.stats import ttest_ind


In [ ]:
sys.path

In [ ]:
pvals = np.zeros((len(grouped),len(grouped)))
for i,g in enumerate(grouped.groups):
    for j,g2 in enumerate(grouped.groups):
        pvals[i,j] = ttest_ind(grouped.get_group(g)['top1'],grouped.get_group(g2)['top1'])[1]

In [ ]:
from matplotlib.colors import LogNorm

plt.figure(figsize=(10,10))
sns.heatmap(pvals,xticklabels=grouped.groups,yticklabels=grouped.groups,norm=LogNorm(vmin=pvals.min()+0.00000001, vmax=pvals.max()))

In [ ]:
for g2 in grouped.groups:
    g = (g2[0],g2[1],'1 eranked 40')
    print(g2,ttest_ind(grouped.get_group(g)['top1'],grouped.get_group(g2)['top1'])[1])

In [ ]:
g2